In [1]:
import agate

tester = agate.TypeTester(limit=100)
fiftyyears = agate.Table.from_csv('../data/temp1.csv', column_types=tester)

In [2]:
print (fiftyyears)

|---------------+---------------|
|  column_names | column_types  |
|---------------+---------------|
|  STATION      | Text          |
|  STATION_NAME | Text          |
|  ELEVATION    | Number        |
|  LATITUDE     | Number        |
|  LONGITUDE    | Number        |
|  Date         | Date          |
|  TMAX         | Number        |
|  TMIN         | Number        |
|---------------+---------------|



In [5]:
from decimal import Decimal

def make_high_f(row): 
    return ((row['TMAX']/10)*Decimal(1.8)+32).quantize(Decimal('0.1'))

def make_low_f(row): 
    return ((row['TMIN']/10)*Decimal(1.8)+32).quantize(Decimal('0.1'))
            
converted_temps = fiftyyears.compute([
        ('tmax_f', agate.Formula(agate.Number(), make_high_f)),
        ('tmin_f', agate.Formula(agate.Number(), make_low_f))
    ])


In [6]:
print(converted_temps)

|---------------+---------------|
|  column_names | column_types  |
|---------------+---------------|
|  STATION      | Text          |
|  STATION_NAME | Text          |
|  ELEVATION    | Number        |
|  LATITUDE     | Number        |
|  LONGITUDE    | Number        |
|  Date         | Date          |
|  TMAX         | Number        |
|  TMIN         | Number        |
|  tmax_f       | Number        |
|  tmin_f       | Number        |
|---------------+---------------|



In [7]:
temps_with_months = converted_temps.compute([
        ('month', agate.Formula(agate.Text(), lambda row: '%s' % row['Date'].month))
    ])

In [9]:
print(temps_with_months)
print(len(temps_with_months.rows))

|---------------+---------------|
|  column_names | column_types  |
|---------------+---------------|
|  STATION      | Text          |
|  STATION_NAME | Text          |
|  ELEVATION    | Number        |
|  LATITUDE     | Number        |
|  LONGITUDE    | Number        |
|  Date         | Date          |
|  TMAX         | Number        |
|  TMIN         | Number        |
|  tmax_f       | Number        |
|  tmin_f       | Number        |
|  month        | Text          |
|---------------+---------------|

43551


In [10]:
airport = temps_with_months.where(lambda row: row['STATION'] == 'GHCND:USW00014939')
print(len(airport.rows))

15872


In [11]:
airport_feb = airport.where(lambda row: row['month'] == '2')
print(len(airport_feb.rows))

1228


In [12]:
warm_feb_days = airport_feb.where(lambda row: 60 <= row['tmax_f'] < 70)
print(len(warm_feb_days.rows))

88


In [16]:
binned_temps = airport_feb.bins('tmax_f', 11, -20, 90)
binned_temps.print_bars('tmax_f', 'Count', width=50)

tmax_f    Count
[-10 - 0)     3 ▓                                 
[0 - 10)     22 ▓░░                               
[10 - 20)   103 ▓░░░░░░░░                         
[20 - 30)   203 ▓░░░░░░░░░░░░░░░░░                
[30 - 40)   355 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░    
[40 - 50)   233 ▓░░░░░░░░░░░░░░░░░░░              
[50 - 60)   199 ▓░░░░░░░░░░░░░░░░                 
[60 - 70)    88 ▓░░░░░░░                          
[70 - 80)    22 ▓░░                               
                +-------+-------+--------+-------+
                0      100     200      300    400


In [17]:
warm_feb_days_with_years = warm_feb_days.compute([
        ('year', agate.Formula(agate.Text(), lambda row: '%s' % row['Date'].year))
    ])

In [18]:
by_year = warm_feb_days_with_years.group_by('year')

In [22]:
by_year_count = by_year.aggregate([
        ('count', agate.Count())
    ])

In [23]:
by_year_count.print_bars('year', 'count', width=50)

year count
1974     2 ▓░░░░░░░░░░                            
1976     6 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░          
1977     8 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░
1981     6 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░          
1982     2 ▓░░░░░░░░░░                            
1983     1 ▓░░░░░                                 
1984     3 ▓░░░░░░░░░░░░░░                        
1986     2 ▓░░░░░░░░░░                            
1987     4 ▓░░░░░░░░░░░░░░░░░░░                   
1988     1 ▓░░░░░                                 
1990     3 ▓░░░░░░░░░░░░░░                        
1991     3 ▓░░░░░░░░░░░░░░                        
1992     5 ▓░░░░░░░░░░░░░░░░░░░░░░░░              
1994     2 ▓░░░░░░░░░░                            
1995     3 ▓░░░░░░░░░░░░░░                        
1996     3 ▓░░░░░░░░░░░░░░                        
1997     2 ▓░░░░░░░░░░                            
1998     1 ▓░░░░░                                 
1999     4 ▓░░░░░░░░░░░░░░░░░░░                   
2000     7 ▓░░░░░░░░

In [24]:
average_february = airport_feb.compute([
        ('year', agate.Formula(agate.Text(), lambda row: '%s' % row['Date'].year))
    ])

In [31]:
average_feb_by_year = average_february.group_by('year')

In [32]:
average_feb_by_year = average_feb_by_year.aggregate([
        ('average', agate.Mean('tmax_f'))
    ])

In [33]:
average_feb_by_year.print_bars('year', 'average', width=80)

year                       average
1973 36.80000000000000000000000000 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░                 
1974 41.85357142857142857142857143 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░             
1975 28.27857142857142857142857143 ▓░░░░░░░░░░░░░░░░░░░░░                       
1976 50.33793103448275862068965517 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░       
1977 47.46785714285714285714285714 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░         
1978 22.75000000000000000000000000 ▓░░░░░░░░░░░░░░░░░                           
1979 23.27857142857142857142857143 ▓░░░░░░░░░░░░░░░░░                           
1980 30.73448275862068965517241379 ▓░░░░░░░░░░░░░░░░░░░░░░░                     
1981 45.22500000000000000000000000 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░           
1982 33.10000000000000000000000000 ▓░░░░░░░░░░░░░░░░░░░░░░░░                    
1983 39.96071428571428571428571429 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░               
1984 46.35517241379310344827586207 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░    